# Création sécurisée de engine  
engine = la connexion physique à ta base (SQLite, Postgres…).  
Base = la fabrique de classes ORM (elle contient le MetaData).  

👉 Il n’y a pas de lien direct entre engine et Base.  
Ils se rejoignent seulement quand tu demandes au MetaData de créer les tables sur un engine

In [1]:
from pathlib import Path
from sqlalchemy import create_engine,text,__version__
from sqlalchemy.orm import declarative_base

rep_defaut = Path.cwd().parent.parent
print(f"VC_1 ---> Répertoire par défaut {rep_defaut}")
print(f"VC_2 ---> Version SQLAlchemy {__version__}")

rep_source = rep_defaut / "sources"

rep_bdd = rep_defaut / "bdd" / "essais.sqlite"
print(f"VC_3 ---> Répertoire de la BDD {rep_bdd.as_posix()}")


try:
    DATABASE_URL = f"sqlite:///{rep_bdd.as_posix()}"
    engine = create_engine(DATABASE_URL)
    # Test de connexion (force SQLite à créer le fichier)
    with engine.begin() as conn:
        conn.execute(text("SELECT 1"))
    res = "a été créée" if rep_bdd.exists() else "n'a pas été créée"
    print(f"VC_4 ---> La  bdd {res}")
    Base = declarative_base()
    print(f"VC_5 ---> Création de Base réussie")
except Exception as e:
    print(f"VC_4 ---> Erreur lors de la création de l'engine : {e}")
    print(f"VC_5 ---> Erreur lors de la création de Base : {e}")

VC_1 ---> Répertoire par défaut p:\Yvon\AAAAA-Mes-donnees\Mes-developpements\Python_tous_mes_tutoriels\projet_rgd_pas_a_pas
VC_2 ---> Version SQLAlchemy 2.0.41
VC_3 ---> Répertoire de la BDD p:/Yvon/AAAAA-Mes-donnees/Mes-developpements/Python_tous_mes_tutoriels/projet_rgd_pas_a_pas/bdd/essais.sqlite
VC_4 ---> La  bdd a été créée
VC_5 ---> Création de Base réussie


# Création et peuplement d'une table à partir de ORM

In [34]:
from sqlalchemy import MetaData, delete, inspect, Table, Column, Integer, String
from sqlalchemy.orm import declarative_base, Session
import pandas as pd

def synoptique_1(engine, Base):
    insp = inspect(engine)
    metadata = maj_metadata()
    
    # Tables réelles dans SQLite
    tables_reelles = set(insp.get_table_names())

    # Classes mappées -> Tables Python
    classes_mappees = {
        mapper.class_.__name__: mapper.local_table
        for mapper in Base.registry.mappers
    }

    # Tables Python présentes dans metadata
    tables_python = metadata.tables

    # Ensemble global de tous les noms de tables
    tout = set(tables_reelles) | set(tables_python.keys()) | {t.name for t in classes_mappees.values()}

    lignes = []
    for nom_table in sorted(tout):
        # Classe associée
        classe_associee = next(
            (cls for cls, table in classes_mappees.items() if table.name == nom_table),
            None
        )
        # Objet Python Table
        table_python = tables_python.get(nom_table)

        # Détermination du statut
        if nom_table in tables_reelles and classe_associee is not None and table_python is not None:
            statut = "Complet ✅"
        elif nom_table not in tables_reelles:
            statut = "Table réelle absente ❌"
        elif classe_associee is None:
            statut = "Classe manquante ❌"
        elif table_python is None:
            statut = "Objet Python Table manquant ❌"
        else:
            statut = "Incohérence ⚠️"


        lignes.append({
            "Table réelle SQLite3": nom_table if nom_table in tables_reelles else None,
            "Classe mappée": classe_associee,
            "Objet Python Table": table_python,
            "Statut global": statut
        })

    df = pd.DataFrame(lignes)

    # Fonction de style pour coloriser
    def coloriser(val):
        if val is None or "❌" in str(val):
            return "background-color: #f8d7da;"  # rouge clair
        elif "✅" in str(val):
            return "background-color: #d4edda;"  # vert clair
        elif "⚠️" in str(val):
            return "background-color: #fff3cd;"  # jaune clair
        else:
            return ""

    return df.style.applymap(coloriser, subset=df.columns)

def synoptique(engine, Base):
    insp = inspect(engine)
    metadata = maj_metadata()
    
    # Tables réelles dans SQLite
    tables_reelles = set(insp.get_table_names())

    # Classes mappées -> Tables Python
    classes_mappees = {
        mapper.class_.__name__: mapper.local_table
        for mapper in Base.registry.mappers
    }

    # Tables Python présentes dans metadata
    tables_python = metadata.tables

    # Ensemble global de tous les noms de tables
    tout = set(tables_reelles) | set(tables_python.keys()) | {t.name for t in classes_mappees.values()}

    lignes = []
    for nom_table in sorted(tout):
        # Classe associée
        classe_associee = next(
            (cls for cls, table in classes_mappees.items() if table.name == nom_table),
            None
        )
        # Objet Python Table
        table_python = tables_python.get(nom_table)

        lignes.append({
            "Table réelle SQLite3": nom_table if nom_table in tables_reelles else None,
            "Classe mappée": classe_associee,
            "Objet Python Table": table_python
        })

    return pd.DataFrame(lignes)

def inventaire_dict(engine, Base):
    """
    Retourne un dict de la forme :
    {
        "Nom": {
            "classe_orm": <classe ou None>,
            "table_reelle": <nom table BDD ou None>,
            "table_python": <objet Table ou None>
        },
        ...
    }
    """
    insp = inspect(engine)

    # Tables réelles
    tables_reelles = set(insp.get_table_names())

    # Classes ORM mappées
    classes_mappees = {
        mapper.class_.__name__: mapper.class_
        for mapper in Base.registry.mappers
    }
    classes_vers_tables = {
        mapper.class_.__name__: mapper.local_table.name
        for mapper in Base.registry.mappers
    }

    # Tables Python (metadata)
    maj_metadata()
    tables_python = Base.metadata.tables

    # Fusion de tous les noms possibles
    noms = set(classes_mappees.keys()) | set(classes_vers_tables.values()) | tables_reelles | set(tables_python.keys())

    resultat = {}
    for nom in sorted(noms):
        resultat[nom] = {
            "classe_orm": classes_mappees.get(nom, None),
            "table_reelle": nom if nom in tables_reelles else None,
            "table_python": tables_python.get(nom, None),
        }

    return resultat

def inventaire_complet(engine, Base):
    """
    Retourne un DataFrame listant :
    - les classes ORM
    - les tables réelles en BDD
    - les objets Python Table
    - la correspondance Classe ↔ Table
    """
    insp = inspect(engine)

    # 1. Tables réelles en base
    tables_reelles = set(insp.get_table_names())

    # 2. Classes ORM mappées (Classe → Table)
    classes_mappees = {
        mapper.class_.__name__: mapper.local_table.name
        for mapper in Base.registry.mappers
    }

    # 3. Objets Python Table connus de metadata
    maj_metadata()
    tables_python = set(Base.metadata.tables.keys())

    # Fusion des noms possibles
    noms = set(classes_mappees.keys()) | set(classes_mappees.values()) | tables_reelles | tables_python

    rows = []
    for nom in sorted(noms):
        # Existence dans chaque monde
        est_classe = nom in classes_mappees.keys()
        est_table_reelle = nom in tables_reelles
        est_table_python = nom in tables_python

        # Correspondance éventuelle Classe ↔ Table
        correspondance = None
        if est_classe:
            correspondance = f"{nom} ↔ {classes_mappees[nom]}"

        rows.append({
            "Nom": nom,
            "Classe ORM": "✅" if est_classe else "❌",
            "Table réelle (BDD)": "✅" if est_table_reelle else "❌",
            "Objet Python Table": "✅" if est_table_python else "❌",
            "Correspondance": correspondance or "-"
        })

    return pd.DataFrame(rows)

def lister_tables_reelles():
    insp = inspect(engine)
    # Tables réelles
    return set(insp.get_table_names())
    
def lister_classes_mappees(Base):
    return {
        mapper.class_.__name__: mapper.class_
        for mapper in Base.registry.mappers
    }

def lister_tables_python():
    """Attention, ces tables python existent dès que la classe ORM est définie, même si elles ne sont pas encore créées en base."""
    return {
        mapper.class_.__name__: mapper.local_table.name
        for mapper in Base.registry.mappers
    }
        
def inventaire_1(engine, Base):
    """
    Liste l'état complet des classes mappées et des tables réelles.
    Retourne un dict {nom: etat}.
    """
    insp = inspect(engine)

    # Tables réelles en base
    tables_reelles = set(insp.get_table_names())

    # Classes ORM mappées
    classes_mappees = {
        mapper.class_.__name__: mapper.local_table.name
        for mapper in Base.registry.mappers
    }

    # Fusion des noms : classes + tables
    noms = set(tables_reelles) | set(classes_mappees.keys()) | set(classes_mappees.values())

    resultat = {}
    for nom in noms:
        est_classe = nom in classes_mappees or any(nom == table for table in classes_mappees.values())
        est_table = nom in tables_reelles

        if est_classe and est_table:
            etat = "Classe ORM + Table réelle"
        elif est_classe:
            etat = "Classe ORM seulement"
        elif est_table:
            etat = "Table réelle seulement"
        else:
            etat = "Inexistant"  # normalement impossible ici

        resultat[nom] = etat

    return resultat

def maj_metadata():
    global metadata
    metadata = MetaData()
    metadata.reflect(bind=engine)
    print("Metadata mis à jour")
    return metadata

def vider_table(nom_table):
    if nom_table in lister_tables_reelles():
        table = metadata.tables[nom_table]
        with engine.begin() as conn:
            conn.execute(delete(table))  # supprime toutes les lignes
        print(f"La table '{nom_table}' a été vidée.")
    else:
        print(f"La table '{nom_table}' n'existe pas.")

def supprimer_table(nom_table):
    global metadata
    maj_metadata()
    if nom_table in metadata.tables:
        table = metadata.tables[nom_table]
        table.drop(engine, checkfirst=True)
        metadata = maj_metadata()
        print(f"La table '{nom_table}' a été supprimée.")
    else:
        print(f"La table '{nom_table}' n'existe pas.")

def supprimer_toutes_tables():
    global metadata
    maj_metadata()
    metadata.drop_all(engine)
    metadata = maj_metadata()
    print("Toutes les tables ont été supprimées.")

supprimer_toutes_tables()
Base = declarative_base()
# 1) Mise à jour de metadata
metadata = maj_metadata()

# 2) Suppression de la table utilisateur si elle existe
supprimer_table("utilisateur")

# 3) Présentation globale de la structure actuelle
print(f"Liste complète des classes mappées et des tables réelles : {inventaire_1(engine, Base)}")

# Analyse de la configuration des trois mondes:
#            Classe ORM mappée (dans Base.registry.mappers) 
#            Table réelle (dans la base SQLite via inspect)
#            Objet Python Table (dans Base.metadata.tables)
print(inventaire_dict(engine, Base))

# 4) Définition de la table utilisateur via une classe ORM
if "utilisateur" in lister_tables_reelles():
    print("La classe Utilisateur existe")
else:
    class UtilisateurMachine(Base):
        __tablename__ = "t_utilisateur_machine"

        id = Column(Integer, primary_key=True, autoincrement=True)
        nom = Column(String, nullable=False)
        age = Column(Integer)
        def __repr__(self):
            return f"<UtilisateurMachine(id={self.id}, nom='{self.nom}', age={self.age})>"
    # Création effective de la table dans la base
    Base.metadata.create_all(engine)
    maj_metadata()  # <-- Ajouté pour rafraîchir metadata après création de la table
    print("La classe UtilisateurMachine a été créée")

# 5) Vider la table utilisateur si elle existe
vider_table("t_utilisateur_machine")

# 6) Peupler la table - Ouverture d’une session et insertion de données
with Session(engine) as session:
    # Création d'objets Python
    u1 = UtilisateurMachine(nom="Alice", age=30)
    u2 = UtilisateurMachine(nom="Bob", age=25)
    u3 = UtilisateurMachine (nom="Charlie", age=40)

    # Ajout dans la session
    session.add_all([u1, u2, u3])
    session.commit()

# 7) Lecture des données insérées
with Session(engine) as session:
    utilisateurs = session.query(UtilisateurMachine).all()
    # Validation (INSERT en base)
    session.commit()
    for u in utilisateurs:
        print(u)
    
# 8) Inventaire des feuilles du classeur source_test.xlsm, puis conversion de chacune d'elles en table de la bdd
supprimer_table("t_feuille_test")
classeur = rep_source / "source_test.xlsm"
liste_feuilles = pd.ExcelFile(classeur).sheet_names
print(f"Feuilles dans source_test.xlsm : {liste_feuilles}")

for nom_feuille in liste_feuilles:
    try: 
        df = pd.read_excel(classeur, sheet_name=nom_feuille)
        # Transformer la feuille en table SQL
        try:
            nom_table = "t_" + nom_feuille.lower()
            df.to_sql(nom_table, con=engine, index=True,
                    index_label='id', if_exists="replace")
        except:
            pass
    except Exception as e:
        pass

# 9) Présentation globale de la structure actuelle
# print(f"Liste complète des classes mappées et des tables réelles : {inventaire_dict(engine, Base)}")
# inventaire_complet(engine, Base)
# synoptique(engine, Base)
synoptique_1(engine, Base)


Metadata mis à jour
Metadata mis à jour
Toutes les tables ont été supprimées.
Metadata mis à jour
Metadata mis à jour
La table 'utilisateur' n'existe pas.
Liste complète des classes mappées et des tables réelles : {}
Metadata mis à jour
{}
Metadata mis à jour
La classe UtilisateurMachine a été créée
La table 't_utilisateur_machine' a été vidée.
<UtilisateurMachine(id=1, nom='Alice', age=30)>
<UtilisateurMachine(id=2, nom='Bob', age=25)>
<UtilisateurMachine(id=3, nom='Charlie', age=40)>
Metadata mis à jour
La table 't_feuille_test' n'existe pas.
Feuilles dans source_test.xlsm : ['Feuille_test']
Metadata mis à jour


C:\Users\yvonp\AppData\Local\Temp\ipykernel_20476\2914952969.py:67: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  return df.style.applymap(coloriser, subset=df.columns)


,Table réelle SQLite3,Classe mappée,Objet Python Table,Statut global
0,t_feuille_test,None,t_feuille_test,Classe manquante ❌
1,t_utilisateur_machine,UtilisateurMachine,t_utilisateur_machine,Complet ✅
